<a href="https://colab.research.google.com/github/Oreolorun/Web-Scraping/blob/main/WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#  Importing libraries
from urllib.request import urlopen
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import requests
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [8]:
#  mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **Scraper Class**

In [61]:
class Scraper:
  """
  This class is used to create an image web scraper
  """
  def __init__(self, header):
    self.header = header
  
  def __str__(self):
    return """
    Methods Available:
    .scrape(): extracts tags of interest
    .download_images(): downloads images using src extracted from tags
    .duplicate_check(): checks for and deletes duplicates
    .delete_all(): deletes all instances of a list of images

    """ 
    
    
  def scrape(self, url, tag, attribute_dict, pages=1):
    """
    This method extracts img tags. Inspect to extract src.
    """
    images = []
    for i in tqdm(range(pages)):
      request = urllib.request.Request(os.path.join(url, f'?page={str(i)}'), 
                                       headers= self.header)
      html = urlopen(request)
      bs = BeautifulSoup(html.read(), 'html.parser')
      image_tags = bs.find_all(tag, attrs=attribute_dict)

      for image_tag in image_tags:
        images.append(image_tag)
    
    return images

  def download_images(self, src_list, directory, prefix='img'):
    """
    This method downloads scraped images into a specified directory
    """
    try:
      os.mkdir(directory)
    except FileExistsError:
      image_count = len(os.listdir(directory))
      for src in tqdm(src_list):
        with open(os.path.join(directory, prefix + f'_{str(image_count)}.jpg'), 
                  'wb') as f:
                  response = requests.get(src)
                  f.write(response.content)
        image_count+=1
      
    print('Done!')

  def duplicate_check(self, directory, filenames=[]):
    """
    This method checks if particular images are duplicated providing the option
    of deleting them or not. 
    """
    to_check = []
    #  creating a list to hold duplicates
    all_duplicates = []

    #  Appending duplicated images array to list
    for f in tqdm(filenames):
      instance = cv2.imread(os.path.join(directory, f))
      to_check.append(instance)

    #  looping through all files
    for f in tqdm(os.listdir(directory)):
      #  reading image files
      image_instance = cv2.imread(os.path.join(directory, f))
      #  looping through all images to be checked
      for item in to_check:
        #  comparing arrays 
        check = np.array_equal(image_instance,item)
        if check:
          #  appending duplicate to list if condition holds true
          all_duplicates.append(f)
    

    if len(to_check)==len(all_duplicates):
      print('\nThere are no duplicated instances.')
    else:
      print(f'\nTotal number of duplicates:'+ 
            f' {len(all_duplicates[len(to_check):])}')
   
    REQUEST_INPUT = True

    while REQUEST_INPUT:
      user_input = input('Would you like to delete duplicates? (Yes(y)/No(n)): ')

      if user_input.lower() == 'y':
        for instance in tqdm(all_duplicates[len(to_check):]):
          try:
            os.remove(os.path.join(directory, instance))
          except FileNotFoundError:
            pass
        print('\nDone!')
        REQUEST_INPUT = False
      elif user_input.lower() == 'n':
        print('Done!')
        REQUEST_INPUT = False
      else:
        print('Invalid Input!')

  def delete_all(self, directory, filenames):
    """
    This method deletes all instances of a particular image. 
    """
    to_check = []
    #  creating a list to hold duplicates
    all_duplicates = []

    #  Appending image instance array to list
    for f in tqdm(filenames):
      instance = cv2.imread(os.path.join(directory, f))
      to_check.append(instance)

    #  looping through all files
    for f in tqdm(os.listdir(directory)):
      #  reading image files
      image_instance = cv2.imread(os.path.join(directory, f))
      #  looping through all images to be checked
      for item in to_check:
        #  comparing arrays 
        check = np.array_equal(image_instance,item)
        if check:
          #  appending duplicate to list if condition holds true
          all_duplicates.append(f)
    
    while True:
      user_input = input(f'There are/is {len(all_duplicates)} instances in this dataset.'+
                        "\nConfirm deletion (Confirm(c)/Cancel(x)): ") 
      
      if user_input.lower() == 'c': 
        #  deleting images
        try:
          for instance in all_duplicates:
            os.remove(os.path.join(directory, instance))
        except FileNotFoundError:
          pass
        break
      elif user_input.lower() == 'x':
        pass
        break
      else:
        print('Invalid Input!\n')
    print('\nDone!')

In [62]:
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

url = 'https://www.truecar.com/used-cars-for-sale/listings/body-coupe/location-georgetown-pa/'

attrs = {'class':'img-inner img-block img-crop'}

image_scraper = Scraper(header=header)

In [22]:
tags = image_scraper.scrape(url, 'img', attrs, 4)

100%|██████████| 4/4 [00:06<00:00,  1.68s/it]


In [79]:
src = [x['style'][21:-1] for x in tags]

folder = 'gdrive/My Drive/Datasets/from_scraper'

image_scraper.download_images(src[:5], folder)

100%|██████████| 5/5 [00:00<00:00, 14.03it/s]

Done!


In [81]:
folder_2 = 'gdrive/My Drive/Datasets/Car_Images/sedans'

image_scraper.duplicate_check(folder_2, ['sedan_45202.jpg'])

100%|██████████| 20290/20290 [09:01<00:00, 37.44it/s]



Total number of duplicates: 1
Would you like to delete duplicates? (Yes(y)/No(n)): n
Done!


In [64]:
image_scraper.delete_all(folder, ['img_122.jpg'])

100%|██████████| 119/119 [00:00<00:00, 169.29it/s]


There are/is 1 instances in this dataset.
Confirm deletion (Confirm(c)/Cancel(x)): g
Invalid Input!

There are/is 1 instances in this dataset.
Confirm deletion (Confirm(c)/Cancel(x)): x

Done!


In [54]:
len(os.listdir(folder))

122

## Checking for duplicates

In [ ]:
#  Checking for duplicates
FLAG_30 = False

while FLAG_30:
  #  creating list to hold images
  image_holder = []

  #  appending image array to list
  print('appending images to list...')
  for f in tqdm(os.listdir()):
    f_temp = cv2.imread(f)
    image_holder.append(f_temp)
  
  #  creating a queue of images
  print('populating placeholder list...')
  image_holder_queue = [x for x in image_holder]

  #  creating a list to hold duplicates
  duplicate_element = []
  k = 0

  print('checking for duplicates...')
  #  looping through images
  for element in tqdm(image_holder):
    try:
      image_holder_queue.remove(element)
      for item in image_holder_queue:
        check = np.array_equal(element, item)
        if check:
          for arr in duplicate_element:
            if not np.array_equal(element, arr):
              duplicate_element.append(element)
              k+=1
    except TypeError:
      pass
  print(f'\nThere are {len(duplicate_element)} duplicates in this directory')
  break 

In [73]:
def duplicate_check(directory):
  #  creating empty lists to hold image arrays
  images, images_temp = [], []
  #  creating a list to hold file names
  file_names = []

  #  reading images
  print('reading images...')
  for img_file in tqdm(os.listdir(directory)):
    try:
      f = cv2.imread(os.path.join(directory, img_file))
      f = cv2.resize(f, (30,30)) #  resizing
      file_names.append(img_file) 
      images.append(f) #  appending to list
      images_temp.append(f)
    except Exception:
      pass

  i=0 #  counter
  duplicate_idx = []
  #  looping through images
  print('checking images...')
  for image in tqdm(images):
    images_temp.remove(image)
    for img in images_temp:
      if np.array_equal(image, img):
        duplicate_idx.append(i)
    i+=1
  
  print(f'There are {len(duplicate_idx)} duplicates in the dataset')

  #  deriving duplicate file names
  print('deriving file names...')
  duplicated_instances = []
  for idx in tqdm(duplicate_idx):
    duplicated_instances.append(file_names[idx])
  return duplicated_instances

In [78]:
temp = duplicate_check(folder)

reading images...


100%|██████████| 89/89 [00:00<00:00, 184.14it/s]


checking images...


100%|██████████| 89/89 [00:00<00:00, 2960.20it/s]


There are 0 duplicates in the dataset
deriving file names...


0it [00:00, ?it/s]


In [ ]:
temp